# Import files, libraries, define path and initialize lists

In [ ]:
!pip install kagglehub
from google.colab import files

import kagglehub

# Download the latest version of the dataset
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

100%|██████████| 2.70G/2.70G [02:07<00:00, 22.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2


In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
from PIL import Image
import glob
import os
import shutil
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint

Mounted at /content/drive


In [ ]:
# Define the dataset path
root_folder = "/root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2"

data_dir = os.path.join(root_folder, 'new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/')

test_path = os.path.join(root_folder, 'test/test')
train_path = os.path.join(data_dir, 'train')
valid_path = os.path.join(data_dir, 'valid')

In [ ]:
# Initialize lists
image_paths = []
species_labels = []
disease_labels = []
dataset_split = []

# Function to process a dataset directory
def process_directory(base_path, split_name):
    for class_folder in os.listdir(base_path):
        class_folder_path = os.path.join(base_path, class_folder)
        if os.path.isdir(class_folder_path):
            species, disease = class_folder.split('___')
            for image_name in os.listdir(class_folder_path):
                image_paths.append(os.path.join(class_folder_path, image_name))
                species_labels.append(species)
                disease_labels.append(disease)
                dataset_split.append(split_name)

# Process train and valid
process_directory(train_path, 'train')
process_directory(valid_path, 'valid')

# Create DataFrame
df = pd.DataFrame({
    'image_path': image_paths,
    'species': species_labels,
    'disease': disease_labels,
    'split': dataset_split
})

# Model Training

In [ ]:
import keras
keras.backend.clear_session()

#Define model name
Model_name = 'LLC_VGG16_1Block_unfrozen_v1'

In [ ]:
# Define the image generators for the training and validation set
batch_size = 32
num_classes = 38
img_size = (256, 256)

train_datagen = ImageDataGenerator(rescale=1./255)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
    valid_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


Unfreezing 1 block

In [ ]:
# Import the VGG16 model as a baseline
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(256, 256, 3))

# Print the layers to confirm
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
0 input_layer
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


In [ ]:

for layer in base_model.layers[15:]:  # Unfreeze layers starting from index 15
    layer.trainable = True


In [ ]:


# Add custom layers for classification
model_vgg16 = Sequential()
model_vgg16.add(base_model)
model_vgg16.add(GlobalAveragePooling2D())
model_vgg16.add(Dense(1024, activation="relu"))
model_vgg16.add(Dropout(rate=0.2))
model_vgg16.add(Dense(512, activation="relu"))
model_vgg16.add(Dropout(rate=0.2))
model_vgg16.add(Dense(num_classes, activation="softmax"))

# Compile the model
optimizer = Adam(learning_rate = 0.001)
model_vgg16.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Define callbacks
epoch_checkpoint = ModelCheckpoint(
    'model_epoch_{epoch:02d}.keras',  # Filename to save the model, it will save with the epoch number
    save_weights_only=False,       # Save the entire model (including architecture, optimizer, and weights)
    save_freq='epoch',             # Save the model after every epoch
    verbose=1                      # Print a message when saving the model
)
best_model = ModelCheckpoint(
    f"{Model_name}.keras",
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

callbacks = [
    epoch_checkpoint,
    best_model
]

In [ ]:
# Training the model
model_vgg16.fit(
    train_generator,
    epochs = 50,
    validation_data = validation_generator,
    callbacks = callbacks
    )

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2197/2197 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.1305 - loss: 3.1149
Epoch 1: saving model to model_epoch_01.keras

Epoch 1: val_accuracy improved from -inf to 0.49869, saving model to LLC_VGG16_1Block_unfrozen_v1.keras
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 246s 98ms/step - accuracy: 0.1306 - loss: 3.1146 - val_accuracy: 0.4987 - val_loss: 1.5422
Epoch 2/50
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.6014 - loss: 1.2433
Epoch 2: saving model to model_epoch_02.keras

Epoch 2: val_accuracy improved from 0.49869 to 0.78039, saving model to LLC_VGG16_1Block_unfrozen_v1.keras
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 181s 82ms/step - accuracy: 0.6014 - loss: 1.2432 - val_accuracy: 0.7804 - val_loss: 0.6541
Epoch 3/50
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.7749 - loss: 0.6808
Epoch 3: saving model to model_epoch_03.keras

Epoch 3: val_accuracy improved from 0.78039 to 0.84635, saving model to LLC_VGG16_1Block_unfrozen_v1.keras
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 183s 83ms/

Save model

In [ ]:
# Create the DS_Models folder if it doesn't exist
model_folder = '/content/drive/MyDrive/DS_Models'
os.makedirs(model_folder, exist_ok=True)

# Define the model save path
model_save_path = os.path.join(model_folder, 'LLC_VGG16_1Block_unfrozen_v1.keras')

# Define the path to the model in Colab and the Google Drive save path
colab_model_path = '/content/LLC_VGG16_1Block_unfrozen_v1.keras'  # Adjust if it's located elsewhere in your Colab environment
model_save_path = os.path.join(model_folder, 'LLC_VGG16_1Block_unfrozen_v1.keras')

# Copy the model to Google Drive
!cp {colab_model_path} {model_save_path}

# Confirm the model is saved in Google Drive
!ls /content/drive/MyDrive/DS_Models/

LLC_VGG16_1Block_unfrozen_v1.keras


Load model


In [ ]:
from tensorflow.keras.models import load_model

# Define the path to the model on Google Drive
model_load_path = '/content/drive/MyDrive/DS_Models/LLC_VGG16_1Block_unfrozen_v1.keras'

# Load the model
model_vgg16 = load_model(model_load_path)

# Verify the model is loaded
model_vgg16.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 8, 8, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 38)                  │          19,494 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 47,352,884 (180.64 MB)

 Trainable params: 15,784,294 (60.21 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 31,568,590 (120.42 MB)

Plot metrics

In [ ]:
history = model_vgg16.history

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue', marker='x')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='orange', marker='x')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', color='blue', marker='x')
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange', marker='x')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

AttributeError: 'Sequential' object has no attribute 'history'

# Model Evaluation

Prediction and validation VGG16

In [ ]:
# Generate predictions for validation set
val_pred = model_vgg16.predict(validation_generator)
val_pred_classes = np.argmax(val_pred, axis=1)
true_classes = validation_generator.classes

# Classification report
class_report = classification_report(true_classes, val_pred_classes)
print("Classification Report:\n", class_report)

550/550 ━━━━━━━━━━━━━━━━━━━━ 27s 47ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.03      0.03      0.03       504
           1       0.02      0.02      0.02       497
           2       0.03      0.03      0.03       440
           3       0.03      0.03      0.03       502
           4       0.02      0.02      0.02       454
           5       0.01      0.01      0.01       421
           6       0.02      0.02      0.02       456
           7       0.02      0.02      0.02       410
           8       0.03      0.03      0.03       477
           9       0.04      0.04      0.04       477
          10       0.01      0.01      0.01       465
          11       0.03      0.03      0.03       472
          12       0.03      0.03      0.03       480
          13       0.02      0.02      0.02       430
          14       0.02      0.02      0.02       423
          15       0.02      0.02      0.02       503
          16  

In [ ]:
# Step 1: Evaluate the model on the validation set
loss, accuracy = model_vgg16.evaluate(validation_generator)




550/550 ━━━━━━━━━━━━━━━━━━━━ 28s 48ms/step - accuracy: 0.9611 - loss: 0.1364


In [ ]:
# Step 2: Print the accuracy
print(f'Validation loss: {loss:.2f}')
print(f'Validation accuracy: {accuracy:.2f}')

Validation loss: 0.15
Validation accuracy: 0.96


Evaluate the test folder

In [ ]:
# Define helper function to preprocess a single image
def preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)  # Load image and resize
    img_array = img_to_array(img) / 255.0               # Convert to array and rescale
    return img_array

# Load and preprocess all images from the test folder
def load_test_images(test_path, target_size):
    test_images = []
    filenames = []
    for file in os.listdir(test_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Supported file formats
            file_path = os.path.join(test_path, file)
            test_images.append(preprocess_image(file_path, target_size))
            filenames.append(file)  # Save filenames for reference
    return np.array(test_images), filenames

# Function to check if prediction is correct based on common words
def is_prediction_correct(file_name, predicted_label):
    # Extract meaningful words from file name (split camel case or concatenated words)
    file_name = re.sub(r'\.\w+$', '', file_name)  # Remove file extension
    file_words = set(word.lower() for word in re.findall(r'[A-Z][a-z]*|[a-z]+', file_name))  # Handle camel case and lowercase

    # Extract meaningful words from predicted label
    label_words = set(word.lower() for word in re.findall(r'[A-Za-z]+', predicted_label))  # Convert to lowercase

    # Check if all file words are in the label words
    is_subset = file_words.issubset(label_words)

    # Debugging output to verify
    #print(f"File Words: {file_words}, Label Words: {label_words}, Is Subset: {is_subset}")

    return is_subset

# Parameters
IMG_SIZE = (256, 256)  # Adjust to model's input size
test_path = test_path  # Define the test folder path

# Load and preprocess test images
test_images, filenames = load_test_images(test_path, target_size=IMG_SIZE)

# Make Predictions
predictions = model_vgg16.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Map predictions to class labels
class_labels = train_generator.class_indices
class_indices = {v: k for k, v in class_labels.items()}  # Reverse mapping of class indices
predicted_labels = [class_indices[idx] for idx in predicted_classes]

# Process and evaluate predictions
results = []

for file, pred, label in zip(filenames, predictions, predicted_labels):
    confidence = np.max(pred)  # Get the highest probability (confidence score)
    is_correct = is_prediction_correct(file, label)  # Check prediction correctness
    results.append({"True class": file, "prediction": label, "confidence": confidence, "Correct": int(is_correct)})
    print(f"True class: {file}, Prediction: {label}, Confidence: {confidence:.2f}, Correct: {is_correct}")

# Summary of results
correct_predictions = sum(result['Correct'] for result in results)
total_predictions = len(results)
accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
print(f"Total Predictions: {total_predictions}, Correct: {correct_predictions}, Accuracy: {accuracy:.2%}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
True class: TomatoYellowCurlVirus2.JPG, Prediction: Tomato___Tomato_Yellow_Leaf_Curl_Virus, Confidence: 1.00, Correct: True
True class: TomatoYellowCurlVirus3.JPG, Prediction: Tomato___Tomato_Yellow_Leaf_Curl_Virus, Confidence: 1.00, Correct: True
True class: TomatoEarlyBlight3.JPG, Prediction: Tomato___Early_blight, Confidence: 1.00, Correct: True
True class: TomatoEarlyBlight1.JPG, Prediction: Tomato___Early_blight, Confidence: 1.00, Correct: True
True class: AppleCedarRust3.JPG, Prediction: Peach___Bacterial_spot, Confidence: 0.92, Correct: False
True class: TomatoEarlyBlight2.JPG, Prediction: Tomato___Late_blight, Confidence: 0.72, Correct: False
True class: TomatoYellowCurlVirus6.JPG, Prediction: Tomato___Tomato_Yellow_Leaf_Curl_Virus, Confidence: 1.00, Correct: True
True class: PotatoHealthy2.JPG, Prediction: Potato___healthy, Confidence: 1.00, Correct: True
True class: AppleCedarRust2.JPG, Prediction: Apple___Cedar_apple_rust, Confidence: 0.98

In [ ]:
# Define helper function to preprocess a single image
def preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)  # Load image and resize
    img_array = img_to_array(img) / 255.0               # Convert to array and rescale
    return img_array

# Load and preprocess all images from the test folder
def load_test_images(test_path, target_size):
    test_images = []
    filenames = []
    for file in os.listdir(test_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Supported file formats
            file_path = os.path.join(test_path, file)
            test_images.append(preprocess_image(file_path, target_size))
            filenames.append(file)  # Save filenames for reference
    return np.array(test_images), filenames

# Parameters
IMG_SIZE = (256, 256)  # Adjust to model's input size
test_path = test_path

# Load and preprocess test images
test_images, filenames = load_test_images(test_path, target_size=IMG_SIZE)

# Make Predictions
predictions = model_vgg16.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Map predictions to class labels
class_labels = train_generator.class_indices
class_indices = {v: k for k, v in class_labels.items()}  # Reverse mapping of class indices
predicted_labels = [class_indices[idx] for idx in predicted_classes]

# Print predictions with filenames
results = []

for file, pred, label in zip(filenames, predictions, predicted_labels):
    confidence = np.max(pred)  # Get the highest probability (confidence score)
    print(f"True class:{file}, Predictions:{label} (Confidence: {confidence:.2f})")

    # Pause and ask for feedback
    feedback = input("Is the prediction correct? (1 for Yes, 0 for No): ")
    if feedback == "1":
        results.append({"file": file, "prediction": label, "confidence": confidence, "Correct": "1"})
    else:
        results.append({"file": file, "prediction": label, "confidence": confidence, "Correct": "0"})

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
True class:TomatoYellowCurlVirus2.JPG, Predictions:Tomato___Tomato_Yellow_Leaf_Curl_Virus (Confidence: 1.00)
Is the prediction correct? (1 for Yes, 0 for No): 1
True class:TomatoYellowCurlVirus3.JPG, Predictions:Tomato___Tomato_Yellow_Leaf_Curl_Virus (Confidence: 1.00)
Is the prediction correct? (1 for Yes, 0 for No): 1
True class:TomatoEarlyBlight3.JPG, Predictions:Tomato___Early_blight (Confidence: 1.00)
Is the prediction correct? (1 for Yes, 0 for No): 1
True class:TomatoEarlyBlight1.JPG, Predictions:Tomato___Early_blight (Confidence: 1.00)
Is the prediction correct? (1 for Yes, 0 for No): 1
True class:AppleCedarRust3.JPG, Predictions:Peach___Bacterial_spot (Confidence: 0.92)
Is the prediction correct? (1 for Yes, 0 for No): 0
True class:TomatoEarlyBlight2.JPG, Predictions:Tomato___Late_blight (Confidence: 0.72)
Is the prediction correct? (1 for Yes, 0 for No): 0
True class:TomatoYellowCurlVirus6.JPG, Predictions:Tomato___Tomato_Yellow_Leaf_Curl

Create DataFrame and save as CSV in google drive

In [ ]:
# Create a DataFrame from the results
df_results = pd.DataFrame(results)

# Print the DataFrame
print("\nSummary of Predictions and Feedback:")
display(df_results)

# Save DataFrame to CSV
output_csv_path = f"{save_dir}/{Model_name}_predictions.csv"  # Specify the output file path
df_results.to_csv(output_csv_path, index=False)

print(f"Predictions have been saved to {output_csv_path}")


Summary of Predictions and Feedback:


,file,prediction,confidence,Correct
0,TomatoYellowCurlVirus2.JPG,Tomato___Tomato_Yellow_Leaf_Curl_Virus,1.000000,1
1,TomatoYellowCurlVirus3.JPG,Tomato___Tomato_Yellow_Leaf_Curl_Virus,0.999830,1
2,TomatoEarlyBlight3.JPG,Tomato___Early_blight,0.999221,1
3,TomatoEarlyBlight1.JPG,Tomato___Early_blight,0.999994,1
4,AppleCedarRust3.JPG,Peach___Bacterial_spot,0.922439,0
5,TomatoEarlyBlight2.JPG,Tomato___Late_blight,0.717354,0
6,TomatoYellowCurlVirus6.JPG,Tomato___Tomato_Yellow_Leaf_Curl_Virus,1.000000,1
7,PotatoHealthy2.JPG,Potato___healthy,0.999971,1
8,AppleCedarRust2.JPG,Apple___Cedar_apple_rust,0.982735,0
9,PotatoEarlyBlight5.JPG,Potato___Early_blight,1.000000,1


Predictions have been saved to /content/drive/MyDrive/DS_models/LLC_VGG16_1Block_unfrozen_v1_predictions.csv


prediction accuracy

In [ ]:
df_results["Correct"].value_counts(normalize=True)


,proportion
Correct,
1,0.818182
0,0.181818
